## Read P190
This code reads P190 files. It was originally created by Aditya Karan, UW ECE, in Jan. 2020 and revised by Shima Abadi, UW ECE, in March 2020.  

In [1]:
def readP190(filename):
    import re
    import numpy as np
    
# Function returns the location of all instances of a particular string
    def strOccurences(string, temp):
      return [i.start() for i in re.finditer(string, temp)]

    # Function is used to read group data from VMGL or SMGL or TMGL
    def compute_header(string, mode):
      t1 = string[46:55] 
      t2 = string[55:64]
      t3 = string[64:70]
      if not t3.strip():
        t3 = 0
      if mode == '3':
        return t1, t2, t3
      elif mode == '2':
        return t1, t2

    with open(filename) as file: # Mention the location of the P190 file

    # Calculating Number of Shots
        temp = file.read()
        num = 1
        file.seek(temp.find('H0101'))
        if '2 CABLE' in file.readline():
          num = 2
        file.seek(temp.find('H2600Line'))
        res = [int(i) for i in file.readline().split() if i.isdigit()]
        numShots = res[1] - res[0] + 1
        startNum, endNum = res[0], res[1]

    # Calculating the location of the 'target' string  
        if 'VGL' in temp:
            loc = strOccurences('VGL', temp)
        elif 'VL' in temp:
            loc = strOccurences('VL', temp)
        elif 'VMGL' in temp:
            loc = strOccurences('VMGL', temp)

    # Calculating Groups Per Shots  
        groupsPerShotloc = temp.find('H1100')
        file.seek(groupsPerShotloc)
        groupsPerShot = [int(i) for i in file.readline().split() if i.isdigit()]
        groupsPerShot = groupsPerShot[0]

    # Initializing variables
        class nav:
            shotNumber = np.arange(res[0], res[1] + 1, 1)
            depth = np.zeros(numShots)
            vesselX = np.zeros(numShots)
            vesselY = np.zeros(numShots)
            sourceX = np.zeros(numShots)
            sourceY = np.zeros(numShots)
            tailX = np.zeros(numShots)
            tailY = np.zeros(numShots)
            receiver_depth = np.zeros(numShots)
            receiverX = np.zeros((groupsPerShot, numShots))
            receiverY = np.zeros((groupsPerShot, numShots))

    # Computing variables 
        for i in range(0, numShots):
            file.seek(loc[i])
            str1 = file.readline()
            nav.vesselX[i], nav.vesselY[i], nav.depth[i] = compute_header(str1, '3')
            str2 = file.readline()
            nav.sourceX[i], nav.sourceY[i] = compute_header(str2, '2')

            str3 = file.readline()
            if 'CMGL' in str3:
              str3 = file.readline()
              nav.tailX[i], nav.tailY[i] = compute_header(str3, '2')
              num = 1
            else :
              nav.tailX[i], nav.tailY[i] = compute_header(str3, '2')
            
            if num == 1:
              for j in range(0, groupsPerShot, 3):
                str4 = file.readline()
                # nav.receiverX[j][i], nav.receiverY[j][i], receiver_depth[i] = str4[5:14], str4[14:23], str4[23:27] 
                # nav.receiverX[j + 1][i], nav.receiverY[j + 1][i], receiver_depth[i] = str4[31:40], str4[40:49], str4[49:53]
                # nav.receiverX[j + 2][i], nav.receiverY[j + 2][i], receiver_depth[i] = str4[57:66], str4[66:75], str4[75:79]
                nav.receiverX[j][i], nav.receiverY[j][i] = str4[5:14], str4[14:23] 
                nav.receiverX[j + 1][i], nav.receiverY[j + 1][i] = str4[31:40], str4[40:49]
                nav.receiverX[j + 2][i], nav.receiverY[j + 2][i] = str4[57:66], str4[66:75]
            elif num == 2:
              str4 = file.readline()
              for j in range(0, groupsPerShot, 3):
                str4 = file.readline()
                nav.receiverX[j][i], nav.receiverY[j][i] = str4[5:14], str4[14:23],  str4[23:27]
                nav.receiverX[j + 1][i], nav.receiverY[j + 1][i] = str4[31:40], str4[40:49], str4[49:53]
                nav.receiverX[j + 2][i], nav.receiverY[j + 2][i] = str4[57:66], str4[66:75], str4[75:79]

    return nav